In [1]:
import xarray as xr
import numpy as np
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging

In [2]:
#read the dataset
ds = xr.open_dataset('10minTempStationData.nc')
temp = ds['temperature']
#grid step
step = 0.5

In [3]:
# Y grid
min_lat = np.floor(np.min(ds['lat'].values))
max_lat = np.ceil(np.max(ds['lat'].values)) + step
gridy = np.arange(min_lat, max_lat, step)
# X grid
min_lon = np.floor(np.min(ds['lon'].values))
max_lon = np.ceil(np.max(ds['lon'].values)) + step
gridx = np.arange(min_lon, max_lon, step)

In [4]:
def findIndex(value, value_array):
    min_value = min(value_array, key=lambda x: abs(x - value))
    index = np.where(value_array == min_value)[0][0]
    return index

In [5]:
# handle provided time
input_time = '2010-07-12 13:07:30'
time = np.datetime64(input_time)
time_array = ds['time'].values
time_index = findIndex(time, time_array)

# get the station coordinates
latitudes = ds['lat'].values
longitudes = ds['lon'].values
coordinates = np.vstack((longitudes,latitudes)).T

# create data array
data = []
index = 0
for tuple in coordinates:
    lon = tuple[0]
    lat = tuple[1]
    value = temp[time_index, index].values.item(0)
    index += 1
    if not np.isnan(value):
        data.append([lon, lat, value])

data = np.array(data)

In [6]:
data

array([[ 3.81028008, 55.39920044, 15.19999981],
       [ 6.19606018, 51.49729919, 22.60000038],
       [ 5.94166994, 53.49169922, 18.79999924],
       [ 4.97875977, 52.6427002 , 22.10000038],
       [ 4.92592001, 51.96900177, 23.39999962],
       [ 2.93575001, 54.32569885, 16.20000076],
       [ 5.1796999 , 52.09880066, 24.        ],
       [ 4.78113985, 52.92689896, 20.29999924],
       [ 5.87232018, 52.05490112, 24.29999924],
       [ 6.58484983, 53.12369919, 19.60000038],
       [ 5.37701988, 51.44979858, 25.39999962],
       [ 5.76249981, 51.19670105, 25.70000076],
       [ 4.01221991, 54.11669922, 17.        ],
       [ 4.69610977, 54.85390091, 16.39999962],
       [ 4.93513012, 51.56489944, 25.29999924],
       [ 6.25889015, 52.43439865, 22.39999962],
       [ 5.14540005, 51.85760117, 24.        ],
       [ 4.12185001, 51.99089813, 22.70000076],
       [ 6.5729599 , 52.74909973, 20.        ],
       [ 5.34579992, 53.3913002 , 19.39999962],
       [ 4.15028   , 52.91809845, 17.200

In [7]:
# variogram_model => [linear, power, gaussian, spherical, exponential]
OK = OrdinaryKriging(data[:, 0], data[:, 1], data[:, 2], variogram_model='exponential',
                     verbose=True, enable_plotting=False)
style = 'grid'

if style == 'points':
    z, ss = OK.execute('points', np.array([2.1]), np.array([51.9]))
elif style == 'grid':
    z, ss = OK.execute('grid', gridx, gridy)
else:
    print("Invalid style")

#kt.write_asc_grid(gridx, gridy, z, filename="output.asc")

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'exponential' Variogram Model
Partial Sill: 18.818559010514175
Full Sill: 18.818559010514175
Range: 4.382858513488947
Nugget: 6.54894772825993e-17 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...



In [8]:
if style == 'grid':
    # query location (must be within the limits of the grid)
    query_lon = 2.1
    query_lat = 51.9

    x = findIndex(query_lon, gridx)
    y = findIndex(query_lat, gridy)
    
    print(z[x,y])
elif style == 'points':
    print(z[0])

22.25609450090893
